<a href="https://colab.research.google.com/github/EleonoraBartolomucci/Fairness/blob/master/Faceplusplus/yelp_picture_downloader.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import requests
from lxml import html
import shutil
import os
import json

def downloadUser(user_id, counter):

    folder_id = '1jn_TA3D_mLdPLZJ6C_yL8cwTt7ic5qLA'  # folder ID = face_data/101-
    filename = user_id + '.jpg'
    url = 'https://www.yelp.com/user_details?userid=' + user_id

    try:
        # Find user picture from web page
        page_content = requests.get(url)
        tree = html.fromstring(page_content.content)
        if len(tree.xpath('//*[@id="wrap"]/div[2]/div[1]/div/div[2]/div[1]/div/div/div/a/img/@src')) != 0:  # Picture found
            image_url = tree.xpath('//*[@id="wrap"]/div[2]/div[1]/div/div[2]/div[1]/div/div/div/a/img/@src')[0]

            # View all folders and file in your folder
            fileList = drive.ListFile({'q': "'" + folder_id + "' in parents and trashed=false"}).GetList()

            # Create file on Google Drive
            # param id = ID of Fairness/face_data folder in my Google Drive
            drive_file = drive.CreateFile({'title': filename, 'parents': [{'id': folder_id}]})

            # Check if file already exists in Google Drive (prevents duplicates)
            for file in fileList:
                # print('Title: %s, ID: %s' % (file['title'], file['id']))
                if file['title'] == filename:  # The file already exists, then overwrite it
                    fileID = file['id']
                    drive_file = drive.CreateFile({'id': fileID, 'title': filename, 'parents': [{'id': folder_id}]})

            response = requests.get(image_url, stream=True)

            # Create a local copy of user picture
            with open(filename, 'wb') as out_file:
                shutil.copyfileobj(response.raw, out_file)
            del response

            # Upload user picture on Google Drive
            drive_file.SetContentFile(filename)  # path of local file content
            drive_file.Upload()  # Upload the file.
            # print('title: %s, mimeType: %s' % (drive_file['title'], drive_file['mimeType']))

            # Delete local user pictures
            if os.path.exists(filename):
                os.remove(filename)
            else:
                print("The file does not exist")
    except:
        print("Si è verificato un errore alla riga " + str(counter) + " del file user.json")

In [0]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from oauth2client.client import GoogleCredentials
from google.colab import auth
import json
import csv
import random


# Connect to Google Account
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# DOWNLOAD user.json FROM DRIVE
users_dataset_id = '1aOTHWSNdkjzW7EyyaomdXxgnzjXdqN7S'  # FILE ID, got on google drive with condivision link
download = drive.CreateFile({'id': users_dataset_id})
download.GetContentFile('Tupelo Honey_relevance_desc.csv')

#user_list = lista utenti del ristorante
with open("Tupelo Honey_relevance_desc.csv", 'r') as csv_file:
    csv_reader = csv.reader(csv_file)
    line_count = 0
    user_list = []
    for riga in csv_reader:
      #se è l'header vai alla riga successiva
      if(line_count <= 0):
        line_count = line_count + 1
      else:
        user_list.append(riga[0])


file_list = drive.ListFile({'q': "'1jn_TA3D_mLdPLZJ6C_yL8cwTt7ic5qLA' in parents and trashed=false"}).GetList()

#listaVecchia = lista id utenti già presenti nella cartella
listaVecchia = []
for file1 in file_list:
  id = file1['title'] 
  #elimina '.jpg' 
  listaVecchia.append(id[0:22])


#da user_list elimino gli utenti già processati
for id in listaVecchia :
  user_list.remove(id)

random_user = random.choices(user_list, k=100)

cnt = 0


for user_id in random_user:
  id = '%s' % user_id
  downloadUser(id,cnt)
  cnt = cnt + 1

#for cnt, line in enumerate(fp):
 # json_data = json.loads(line)
  #user_id = json_data['user_id']
  #downloadUser(user_id, cnt)